In [ ]:
import pandas as pd
import csv, re
from bs4 import BeautifulSoup
import urllib3
import requests
import os
os.chdir('D:/programming_files/github/unidata')

In [ ]:
#poolmanager handles all pooling and safety details
http=urllib3.PoolManager()
#the top 200 national universities
url_ranking='https://colleges.niche.com/rankings/best-colleges/'
#header provides a popular browser to open the request
hdr={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11'}
#returns an object that can work as a context manager
r=http.request('GET',url_ranking,headers=hdr,timeout=5)

rank_text=r.data.decode('utf-8')

pattern_rank='"name":"(.+?)"'
regex=re.compile(pattern_rank)
list_rank=regex.findall(rank_text)

with open('top100.csv','w',newline='') as f:
    writer_obj=csv.DictWriter(f,fieldnames=['rank','name','location','accpt_rate','type',
                                           'majors','sat','act','app_fee','tuition','stu_fac','num_undergrd','asian'
                                            ,'internt','avg_slry','grad_rt','emply_rt'])
    writer_obj.writeheader()
    for i in range(0,100):
        writer_obj.writerow({'rank':i+1,'name':list_rank[i+3]})  
#f.close()

unidf=pd.read_csv('top100.csv')
unidf.to_pickle('unidf.pkl')


In [ ]:
unidf=pd.read_pickle('unidf.pkl')

unirows=[]
with open('top100_2.csv','r') as fr:
    reader_obj=csv.reader(fr)
    for row in reader_obj:
        if reader_obj.line_num==1:
            continue
        unirows.append(row)

In [ ]:
url1='https://colleges.niche.com/stanford-university/'
url2='https://colleges.niche.com/stanford-university/cost'
url3='https://colleges.niche.com/stanford-university/students'
r=requests.get(url1)
r2=requests.get(url2)
r3=requests.get(url3)

list_f=[]
pattern_one='"label":"(?:Acceptance Rate|Location Association|SAT Scores Range|ACT Scores Range|Application Fee|Student Faculty Ratio|Full-Time Enrollment)","tooltip":"","value":"?(.+?)"?}'
regex1=re.compile(pattern_one)
list1=regex1.findall(r.text)
pattern_two='"label":"\${ENTITY_NAME} is \.\.\.","tooltip":"","value":"(.+?)"}'
regex2=re.compile(pattern_two)
list2=regex2.findall(r.text)
pattern_three='"name":"Most Popular Majors".+?"name":"(.+?)".+?"name":"(.+?)",.+?"name":"(.+?)"'
regex3=re.compile(pattern_three)
list3=regex3.findall(r.text)
pattern_four='"label":"(?:Graduation Rate|Employed 2 Years After Graduation)","tooltip":".+?","value":(.+?),'
regex4=re.compile(pattern_four)
list4=regex4.findall(r.text)
#/cost
pattern_five='"label":"Out-of-State Tuition","tooltip":"","value":(.+?)\}'
regex5=re.compile(pattern_five)
list5=regex5.findall(r2.text)
#/students
pattern_six='"label":"Racial Diversity.+?Asian":"(.+?)".+?"International.+?":"(.+?)",'
regex6=re.compile(pattern_six)
list6=regex6.findall(r3.text)
#all_features
list_f=list1+list2+list3+list4+list5+[list6[0][0]]+[list6[0][1]]
list_f

In [126]:
#define a function using regex to find needed information
def all_info():
    list_f=[]
    pattern_one='"label":"(?:Acceptance Rate|Location Association|SAT Scores Range|ACT Scores Range|Application Fee|Student Faculty Ratio|Full-Time Enrollment)","tooltip":"","value":"?(.+?)"?}'
    regex1=re.compile(pattern_one)
    list1=regex1.findall(r.text)
    pattern_two='"label":"\${ENTITY_NAME} is \.\.\.","tooltip":"","value":"(.+?)"}'
    regex2=re.compile(pattern_two)
    list2=regex2.findall(r.text)
    pattern_three='"name":"Most Popular Majors".+?"name":"(.+?)".+?"name":"(.+?)",.+?"name":"(.+?)"'
    regex3=re.compile(pattern_three)
    list3=regex3.findall(r.text)
    pattern_four='"label":"(?:Graduation Rate|Employed 2 Years After Graduation)","tooltip":".+?","value":(.+?),'
    regex4=re.compile(pattern_four)
    list4=regex4.findall(r.text)
    #/cost
    pattern_five='"label":"Out-of-State Tuition","tooltip":"","value":(.+?)\}'
    regex5=re.compile(pattern_five)
    list5=regex5.findall(r2.text)
    #/students
    pattern_six='"label":"Racial Diversity.+?Asian":"(.+?)".+?"International.+?":"(.+?)",'
    regex6=re.compile(pattern_six)
    list6=regex6.findall(r3.text)
    #all_features
    list_f=list1+list2+list3+list4+list5+[list6[0][0]]+[list6[0][1]]
    return list_f

In [127]:
requests.packages.urllib3.disable_warnings()
import time
start_time = time.time()
#close warnings
import warnings
warnings.filterwarnings('ignore')

url_start='https://colleges.niche.com/'
tail1='cost/'
tail2='students/'
uni_info=[[]for _ in range(len(unirows))]
dele=[]
for j in range(len(unirows)):
    b=unirows[j][1].replace('-','--')
    url_end=('-'.join(b.split())).lower()+'/'
    urls1=url_start+url_end
    urls2=urls1+tail1
    urls3=urls1+tail2
    try:
        r=requests.get(urls1)
        r2=requests.get(urls2)
        r3=requests.get(urls3)
        uni_info[j].append(all_info())
    except:
        dele.append(j)
        print('The data for '+str(unirows[j][1])+' is neglected')
print("--- %s seconds ---" % (time.time() - start_time))



The data for Washington University in St. Louis is neglected
The data for Washington & Lee University is neglected


The data for College of William & Mary is neglected


The data for Texas A&M University is neglected


The data for Franklin & Marshall College is neglected


--- 117.29170799255371 seconds ---


In [ ]:
import pickle
with open('uni_info.pkl','wb') as f:
    pickle.dump(uni_info,f)

In [ ]:
with open('uni_info.pkl','rb') as f:
    uni_info=pickle.load(f)

In [139]:
dele2=[8, 19, 64, 72, 81]